In [1]:
import pymysql 
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from tqdm import tqdm
from collections import defaultdict, Counter
import _pickle as pickle
import random

In [20]:
# start tunnel first: ssh -f [uni]@mimir.dbmi.columbia.edu -L 3307:127.0.0.1:3306 -N

conn = pymysql.connect(host="127.0.0.1", 
                       user="", #uni
                       port = 59435,
                       passwd="", #sql password
                       db = "clinical_merge_v5_2020q4" ) #database
cur = conn.cursor()

# identify visits

In [3]:
cur.execute('''select distinct pat_mrn_id, date(hosp_admsn_time), max(date(hosp_disch_time))
                from covid_admission
                where char_length(pat_mrn_id) = 10 
                and hosp_disch_time is not NULL 
                and date(hosp_admsn_time) != '0000-00-00'
                and date(hosp_disch_time) != '0000-00-00'
                and (date(hosp_admsn_time) >= '2020-02-01' or date(hosp_disch_time) >= '2020-02-01')
                group by pat_mrn_id, date(hosp_admsn_time);''')

start_visit_id = 0

visit_data = []

for mrn, st, ed in cur.fetchall():
    visit_data.append([start_visit_id, mrn, st, ed])
    start_visit_id += 1
    
visit_data = np.array(pd.DataFrame(visit_data))
    
len(visit_data), len(set(visit_data[:,0])), len(set(visit_data[:,1]))

(1844018, 1844018, 636063)

In [4]:
number_of_visits = Counter(visit_data[:,1])

In [5]:
cur.execute('''select pat_mrn_id, count(distinct date(hosp_admsn_time))
                from covid_admission
                where char_length(pat_mrn_id) = 10 
                and hosp_disch_time is not NULL 
                and date(hosp_admsn_time) != '0000-00-00'
                and date(hosp_disch_time) != '0000-00-00'
                and (date(hosp_admsn_time) >= '2020-02-01' or date(hosp_disch_time) >= '2020-02-01')
                group by pat_mrn_id;''')

visit_cts = {}
for mrn, ct in cur.fetchall():
    visit_cts[mrn] = ct

In [6]:
for mrn in number_of_visits:
    if number_of_visits[mrn] != visit_cts[mrn]:
        print(mrn)

In [7]:
pd.DataFrame(visit_data, columns=['visit_id', 'pat_mrn_id', 'st_dt', 'ed_dt']).to_csv('data/visits_data.csv', index=False)

## load visit_data into mysql database table named vfinal_1_predict_covid_visit 

In [8]:
cur.execute('''select distinct visit_id, sex_c, race_1, race_2, race_3, ethnicity, birth_date, zip
                from covid_persons
                join user_vr2430.vfinal_1_predict_covid_visits using (pat_mrn_id)
                where birth_date is not null;''')

persons_data = []

for i in cur.fetchall():
    if i[6] == '0000-00-00 00:00:00':
        continue
    if i[7] == None:
        persons_data.append([i[0], i[1], i[2], i[3], i[4], i[5], i[6], i[7]])
    else:
        persons_data.append([i[0], i[1], i[2], i[3], i[4], i[5], i[6], i[7][:5]])
    
persons_data = np.array(pd.DataFrame(persons_data))

len(persons_data), len(set(persons_data[:,0]))

(1573113, 1573113)

In [9]:
cur.execute('''select visit_id, contact_date, icd10_code
                from covid_patients
                join user_vr2430.vfinal_1_predict_covid_visits using (pat_mrn_id)
                where contact_date >= st_dt and contact_date <= ed_dt;''')

conditions = []

for i in cur.fetchall():
    if i[2] == None:
        continue
    if ',' in i[2]:
        for k in i[2].split(','):
            conditions.append([i[0], i[1], k.replace(' ', '')])
    else:
        conditions.append([i[0], i[1], i[2]])
        
conditions = np.array(pd.DataFrame(conditions))

len(conditions), len(set(conditions[:,0])), len(set(conditions[:,2]))

(2390214, 718960, 16220)

In [10]:
cur.execute('''select distinct visit_id
               from covid_patients
               join user_vr2430.vfinal_1_predict_covid_visits using (pat_mrn_id)
               where contact_date >= st_dt and contact_date <= ed_dt
               and icd10_code like '%U07.1%';''')

covid_visit = [i for i, in cur.fetchall()]
len(covid_visit)

9340

In [13]:
positive_visits = list(set(persons_data[:,0])&set(covid_visit))
positive_visits = list(set(positive_visits)&set(visit_data[:,0]))
len(positive_visits)

9340

In [14]:
positive_training_set = random.sample(positive_visits, 4670)
positive_eval_set = list(set(positive_visits)-set(positive_training_set))
len(positive_training_set), len(positive_eval_set)

(4670, 4670)

In [15]:
cur.execute('''select distinct visit_id, pat_mrn_id
               from user_vr2430.vfinal_1_predict_covid_visits
               join covid_measurements using (pat_mrn_id)
               where (proc_id in (400660,400670,400671,400834,400662,400833,400620,400697,400747,400571,400661,400570) 
                       or component_id in (36079,36100,36101,36105,36137,36189,36308,36309, 36072, 36074, 36081, 36176, 36177, 36182, 36183, 36342) 
                       or component_loinc_code = 94500)
               and ord_value in ('Detected', 'POSITIVE', 'Yes', 'DETECTED', 'YES', 'Detected..', 'Postive', 'Positive','Detected A')
               and ordering_date >= st_dt and ordering_date <= ed_dt;''')

covid_pos_test_visit = [list(i) for i in cur.fetchall()]

covid_pos_test_visit = np.array(pd.DataFrame(covid_pos_test_visit))
len(covid_pos_test_visit), len(set(covid_pos_test_visit[:,0])), len(set(covid_pos_test_visit[:,1]))

(18156, 18156, 16550)

In [16]:
cur.execute('''select distinct visit_id, pat_mrn_id
               from user_vr2430.vfinal_1_predict_covid_visits
               join covid_measurements using (pat_mrn_id)
               where (proc_id in (400660,400670,400671,400834,400662,400833,400620,400697,400747,400571,400661,400570) 
                       or component_id in (36079,36100,36101,36105,36137,36189,36308,36309, 36072, 36074, 36081, 36176, 36177, 36182, 36183, 36342) 
                       or component_loinc_code = 94500)
               and ordering_date >= st_dt and ordering_date <= ed_dt
               and ord_value in ('Not Detected', 'NotDetectd', 'Negative', 'SARS-CoV-2 neg', 'Not Detected (External)', 'neg', 'Non detected', 'Not Dectected', 'Undetected', 'Negatvie', 'NOT_DETECT', 'notdetected', 'Not Deteceted', 'ND', 'Note Detected', 'not dtected', 'not  detected', 'non-detected', 'not deted', 'Nogative', 'NOPT DETECTED', 'neative', 'No Detected', 'Non-reactive');''')

covid_neg_test_visit = [list(i) for i in cur.fetchall()]

covid_neg_test_visit = np.array(pd.DataFrame(covid_neg_test_visit))
len(covid_neg_test_visit), len(set(covid_neg_test_visit[:,0])), len(set(covid_neg_test_visit[:,1]))

(238444, 238444, 130750)

In [21]:
cur.execute('''select distinct visit_id, pat_mrn_id
               from covid_patients
               join user_vr2430.vfinal_1_predict_covid_visits using (pat_mrn_Id)
               where contact_date >= st_dt and contact_date <= ed_dt
               and icd10_desc like 'COVID-19 ruled out%';''')

covid_ruled_out = [list(i) for i in cur.fetchall()]
covid_ruled_out = np.array(pd.DataFrame(covid_ruled_out))
len(covid_ruled_out)

168

In [22]:
cur.execute('''select distinct visit_id, pat_mrn_id
               from covid_patients
               join user_vr2430.vfinal_1_predict_covid_visits using (pat_mrn_Id)
               where contact_date >= st_dt and contact_date <= ed_dt
               and (icd10_code like '%Z86.16%' or icd10_code like '%Z86.19%');''')

covid_history = [list(i) for i in cur.fetchall()]
covid_history = np.array(pd.DataFrame(covid_history))
len(covid_history)

901

In [23]:
mrn_positive_visits = []
for visit_id in tqdm(positive_visits):
    mrn_positive_visits.append(visit_data[:,1][visit_data[:,0]==visit_id][0])
    
len(mrn_positive_visits), len(set(mrn_positive_visits))

100%|██████████| 9340/9340 [02:58<00:00, 52.27it/s]


(9340, 7625)

In [24]:
visits_mrn_pos_neg = []

for pat_mrn_id in tqdm(set(mrn_positive_visits)):
    visits_mrn_pos_neg += list(visit_data[:,0][visit_data[:,1]==pat_mrn_id])

100%|██████████| 7625/7625 [02:26<00:00, 52.02it/s]


In [25]:
negative_visits = (set(visit_data[:,0])-set(positive_visits))
negative_visits = (set(negative_visits)-set(covid_pos_test_visit[:,0]))
negative_visits = (set(negative_visits)-set(covid_history[:,0]))
negative_visits = (set(negative_visits)-set(visits_mrn_pos_neg))
negative_visits = list(set(negative_visits)&set(persons_data[:,0]))
len(negative_visits)

1483397

In [26]:
negative_training_set = random.sample(negative_visits, 4670)
negative_eval_set = random.sample(list(set(negative_visits)-set(negative_training_set)),4670)

len(negative_training_set), len(negative_eval_set)

(4670, 4670)

In [27]:
demographic_variables = ['age_child', 'age_adol', 'age_adult', 'age_senior', 'sex_1', 'AMERICAN INDIAN OR ALASKA', 
                         'ASIAN', 'BLACK OR AFRICAN AMERICAN', 'NAT.HAWAIIAN/OTH.PACIFIC ', 'WHITE', 
                         'HISPANIC OR LATINO OR SPANISH ORIGIN']

In [28]:
training_and_eval_set = positive_training_set+positive_eval_set+negative_training_set+negative_eval_set
len(training_and_eval_set), len(set(training_and_eval_set))

(18680, 18680)

In [29]:
demographic_matrix = defaultdict(list)

for visit_id, sex_c, race_1, race_2, race_3, ethnicity, birth_date, zip_code in tqdm(persons_data):
    if visit_id not in training_and_eval_set:
        continue
    array_line = []
    st_dt = visit_data[:,2][visit_data[:,0]==visit_id][0]
    age = (st_dt-birth_date.date()).days/365.25
    if age < 13:
        array_line += [1,0,0,0]
    elif age < 19:
        array_line += [0,1,0,0]
    elif age < 60:
        array_line += [0,0,1,0]
    else:
        array_line += [0,0,0,1]
    if sex_c == 1:
        array_line.append(1)
    else:
        array_line.append(0)
    for race in demographic_variables[5:10]:
        if race in [k for k in [race_1, race_2, race_3] if k != None]:
            array_line.append(1)
        else:
            array_line.append(0)
    if ethnicity == 'HISPANIC OR LATINO OR SPANISH ORIGIN':
        array_line.append(1)
    else:
        array_line.append(0)
    demographic_matrix[visit_id]=array_line

100%|██████████| 1573113/1573113 [16:28<00:00, 1590.67it/s]


In [30]:
datetime_variables = ['2020-02-01', '2020-03-01', '2020-04-01', '2020-05-01', '2020-06-01', '2020-07-01', 
                      '2020-08-01', '2020-09-01', '2020-10-01', '2020-11-01', '2020-12-01', '2021-01-01', 
                      '2021-02-01', '2021-03-01', '2021-04-01', '2021-05-01', '2021-06-01', '2021-07-01', 
                      '2021-08-01', '2021-09-01', '2021-10-01', '2021-11-01', '2021-12-01', '2022-01-01',
                      '2022-02-01', '2022-03-01', '2022-04-01']

datetime_variables = [datetime.strptime(i, '%Y-%m-%d').date() for i in datetime_variables]

In [31]:
datetime_matrix = defaultdict(list)

for visit_id, sex_c, race_1, race_2, race_3, ethnicity, birth_date, zip_code in tqdm(persons_data):
    if visit_id not in positive_training_set and visit_id not in negative_training_set and visit_id not in positive_eval_set and visit_id not in negative_eval_set:
        continue
    array_line = []
    st_dt = (visit_data[:,2][visit_data[:,0]==visit_id][0])
    for m in range(len(datetime_variables)-1):
        lo_rg = datetime_variables[m]
        up_rg = datetime_variables[m+1]
        if st_dt >= lo_rg and st_dt < up_rg:
            array_line.append(1)
        else:
            array_line.append(0)
    datetime_matrix[visit_id]=array_line

100%|██████████| 1573113/1573113 [16:40<00:00, 1572.41it/s]


In [32]:
Counter([len(i) for i in demographic_matrix.values()]), len(demographic_variables)

(Counter({11: 18680}), 11)

In [33]:
Counter([len(i) for i in datetime_matrix.values()]), len(datetime_variables)

(Counter({26: 18680}), 27)

In [34]:
len(set(demographic_matrix.keys())), len(set(datetime_matrix.keys()))

(18680, 18680)

In [35]:
diag_variables = list(set([i.split('.')[0] for i in set(conditions[:,2])]))

In [36]:
[i for i in set(conditions[:,2]) if 'U07' in i]

['U07.0', 'U07.1']

In [37]:
diag_matrix = defaultdict(list)

for visit_id, sex_c, race_1, race_2, race_3, ethnicity, birth_date, zip_code in tqdm(persons_data):
    if visit_id not in training_and_eval_set:
        continue
    array_line = []
    diags = set(conditions[:,2][conditions[:,0]==visit_id])
    diags = [k.split('.')[0] for k in diags if k != 'U07.1']
    for d in diag_variables:
        if d in diags:
            array_line.append(1)
        else:
            array_line.append(0)
    diag_matrix[visit_id]=array_line
    

100%|██████████| 1573113/1573113 [18:15<00:00, 1435.87it/s]


In [38]:
Counter([len(i) for i in diag_matrix.values()]), len(diag_variables)

(Counter({1600: 18680}), 1600)

In [39]:
pickle.dump(demographic_matrix, open('data/demographic_matrix.p', 'wb'))
pickle.dump(datetime_matrix, open('data/datetime_matrix.p', 'wb'))
pickle.dump(diag_matrix, open('data/diag_matrix.p', 'wb'))
pickle.dump(demographic_variables, open('data/demographic_variables.p', 'wb'))
pickle.dump(datetime_variables, open('data/datetime_variables.p', 'wb'))
pickle.dump(diag_variables, open('data/diag_variables.p', 'wb'))


In [40]:
pickle.dump(visit_data, open('data/visit_data.p', 'wb'))
pickle.dump(persons_data, open('data/persons_data.p', 'wb'))
pickle.dump(conditions, open('data/conditions.p', 'wb'))
pickle.dump(covid_visit, open('data/covid_visit.p', 'wb'))
pickle.dump(positive_visits, open('data/positive_visits.p', 'wb'))
pickle.dump(covid_pos_test_visit, open('data/covid_pos_test_visit.p', 'wb'))
pickle.dump(covid_neg_test_visit, open('data/covid_neg_test_visit.p', 'wb'))
pickle.dump(negative_visits, open('data/negative_visits.p', 'wb'))
pickle.dump(positive_training_set, open('data/positive_training_set.p', 'wb'))
pickle.dump(negative_training_set, open('data/negative_training_set.p', 'wb'))
pickle.dump(positive_eval_set, open('data/positive_eval_set.p', 'wb'))
pickle.dump(negative_eval_set, open('data/negative_eval_set.p', 'wb'))

In [41]:
len(set(conditions[:,2]))

16220